In [1]:
import pandas as pd
import json
import os
import gzip
from ftplib import FTP
# from bs4 import BeautifulSoup
# import requests


In [2]:
# # ftp.close()
# ftp = FTP('ftp.1000genomes.ebi.ac.uk')
# ftp.login()
# print(ftp.nlst())
# # path to FASTQ
# ftp.cwd('vol1/ftp/phase3/data/HG00096/sequence_read')
# # ftp.cwd('data_collections')
# file_lst = ftp.nlst()
# file_lst

In [3]:
# donwload(file_lst[0], ftp)

In [4]:
# ftp = FTP('ftp.1000genomes.ebi.ac.uk')
# ftp.login()
# # path to BAM
# ftp.cwd('vol1/ftp/phase3/data/HG00096/alignment')
# # ftp.cwd('data_collections')
# ftp.nlst()



In [5]:
# # ftp = FTP('ftp.1000genomes.ebi.ac.uk')
# # ftp.login()
# # ftp.cwd('vol1/ftp/release/20130502')
# # vcf_files = ftp.nlst()

# # # only keep vcf.gz / vcf.gz.tbi
# # vcf_files = [i for i in vcf_files if (i[-3:] == '.gz')]
# # vcf_files

# ftp = FTP('ftp.1000genomes.ebi.ac.uk')
# ftp.login()
# # path to BAM
# ftp.cwd('vol1/ftp/phase3/data/HG00096/alignment')
# # ftp.cwd('data_collections')
# all_files = ftp.nlst()
# bam_files = [i for i in all_files if (i[-4:] == '.bam')]
# bam_files

In [6]:
# ftp.retrlines('LIST')
# ftp.dir()
# ftp.nlst()

In [7]:
# cfg = json.load(open('data-params.json'))
# cfg

In [8]:
# def unzip_gz(path):
#     import gzip
#     for l in gzip.open(path, 'rt'):
#         yield eval(l)

# Assignment

In [59]:
def get_vcf(chromosome, ftp, out_path):
    '''
    chromosome: list of chromosomes needed eg.
    ftp: ftp ojject from the parent func
    '''
    ### chr*
    ### TODO: clean chromosome format
    VCF_SITE = 'vol1/ftp/release/20130502'
    CHR_IDX = 1
    VCF_EXT = '.vcf.gz'
    VCF_IDX = - len(VCF_EXT)
    
    ftp.cwd(VCF_SITE)
    all_files = ftp.nlst()
    # chr* is at the 2nd index place
    # only keep .vcf.gz
    vcf_files = [i for i in all_files if (i[VCF_IDX:] == VCF_EXT)\
                 and (i.split('.')[CHR_IDX] in chromosome)]
     
    out_path = os.path.join(out_path, 'vcf')
    if not (os.path.exists(out_path)):
        os.mkdir(out_path)
    
    donwload(vcf_files, ftp, out_path)
    
    return vcf_files

def get_bam(sample, ftp, out_path):
    '''
    sample: list of samples eg. HG00096
    chromosome: list of chromosomes needed 
    ftp: ftp ojject from the parent func
    '''
    ### chrom*
    BAM_EXT = '.bam'
    BAM_IDX = -len(BAM_EXT)
    ftp.cwd('vol1/ftp/phase3/data')
#     print(ftp.nlst())
    out = []
    for sam in sample:
        BAM_SITE = '%s/alignment' % sam
        
        ftp.cwd(BAM_SITE)
        all_files = ftp.nlst()
        bam_files = [i for i in all_files if (i[BAM_IDX:] == BAM_EXT)]
        
        out_path = os.path.join(out_path, 'bam')
        if not (os.path.exists(out_path)):
            os.mkdir(out_path)
        
        donwload(bam_files, ftp, out_path)
        
        out += bam_files
        ftp.cwd('../../')
        
    return out

def get_fastq(sample, ftp, out_path):
    '''
    ### NO CRHOMOSOME FORMAT
    sample: list of samples eg. HG00096
    chromosome: list of chromosomes needed
    ftp: ftp ojject from the parent func
    '''
    FASTQ_EXT = '.fastq.gz'
    FASTQ_IDX = - len(FASTQ_EXT)
    ftp.cwd('vol1/ftp/phase3/data/')
#     print(ftp.nlst())
    out = []
    for sam in sample:
        FASTQ_SITE = '%s/sequence_read' % sam
        ftp.cwd(FASTQ_SITE)
        all_files = ftp.nlst()
        fastq_files = [i for i in all_files if (i[FASTQ_IDX:] == FASTQ_EXT)]
        
        out_path = os.path.join(out_path, 'fastq')
        if not (os.path.exists(out_path)):
            os.mkdir(out_path)
        donwload(fastq_files, ftp, out_path)
        
        out += fastq_files
        ftp.cwd('../../')
    
    return out


In [66]:
# alignment bam phase1/data/HG00096/
# 'phase3/data/HG00100/sequence_read'


def get_data_list(sample, chromosome, file_format, out_path):
    '''
    file_format either be VCF/BAM/FASTQ
    '''
    SITE = 'ftp.1000genomes.ebi.ac.uk'

    ftp = FTP(SITE)
    ftp.login()
    for i in file_format:
        if 'VCF' == i:
            yield get_vcf(chromosome, ftp, out_path)
            ftp.cwd('../../../../')
            
        elif 'BAM' == i:
            ### TODO determine sample
            yield get_bam(sample, ftp, out_path)
            ftp.cwd('../../../../')

        elif 'FASTQ' == i:
            ### TODO determine sample
            yield get_fastq(sample, ftp, out_path)
            ftp.cwd('../../../../')
            
#     get_data_list(sample, chromosome, file_format, out_path)
    
    ftp.quit()
    ftp.close()


def donwload(file_lst, ftp, out_path):
    '''
    use this funcs at the lst dir
    '''
    for file in file_lst:
        write_path = os.path.join(out_path, file)
        if os.path.exists(write_path):
            print('aready exists' + file)
            continue
        print('downloading %s' % file)
        with open(write_path, 'wb') as fh:
            ftp.retrbinary('RETR %s' % file, fh.write)
        print('finish downloading %s' % file)
#     ftp.cwd('../../')

def get_data(**cfg):
    '''
    reads in the desired data in data-params.json 
    and uses the configuration to download all the tables
    cfg = json.load(open('data-params.json'))
    '''
    
    sample = cfg['sample']
    chrom = cfg['chromosome']
    file_type = cfg['format']
    ###########
    out_path = os.path.join('..', cfg['outpath'])
    
    if not (os.path.exists(out_path)):
        os.mkdir(out_path)
    
    return get_data_list(sample, chrom, file_type, out_path)
    

In [103]:
cfg = json.load(open('../config/data-params.json'))
cfg['chromosome'] = ['chr%s' % s for s in range(1, 23)]
cfg

{'sample': ['HG00096', 'HG00100'],
 'chromosome': ['chr2'],
 'format': ['VCF'],
 'outpath': 'data/raw'}

In [105]:
# for i in get_data(**cfg):
#     print(i)

In [122]:
# for i in get_data_list(['HG00096'], ['chr22'],
#               ['FASTQ']):
#     print(i)

In [147]:
import shlex
import subprocess as sp
def merge_vcf(**cfg):
    lines = ''
    for k, v in cfg['bcftools'].items():
        if isinstance(v, list):
            line = k + ' ' + ' '.join(v)
        elif k == 'concat':
            line = k + ' ' + ' '.join(
                [v.replace('*', 'chr' + str(l)) for l in range(1, 23)]
            ) 
        else:
            line = k + ' ' + v
        lines += line + ' '
    cmd = "bcftools " + lines
    print(cmd)
#     cmd_lst = shlex.split(cmd)
#     proc = sp.call(cmd_lst)



def filter_vcf(**cfg):
    lines = ''
    for k, v in cfg['plink2'].items():
        if isinstance(v, list):
            line = k + ' ' + ' '.join(v)
        else:
            line = k + ' ' + v
        lines += line + ' '
    
    for idx in range(1, 23):
        s = 'chr' + str(idx)
        cmd = ("plink2 " + lines).replace('*', s)
        print(cmd)
#         cmd_lst = shlex.split(cmd)
#         proc = sp.call(cmd_lst)

cfg = json.load(open('../config/pipeline.json')) 
merge_vcf(**cfg)
# cfg = json.load(open('../config/pipeline.json'))
# filter_vcf(**cfg)   

bcftools concat data/interim/chr1_filtered.vcf data/interim/chr2_filtered.vcf data/interim/chr3_filtered.vcf data/interim/chr4_filtered.vcf data/interim/chr5_filtered.vcf data/interim/chr6_filtered.vcf data/interim/chr7_filtered.vcf data/interim/chr8_filtered.vcf data/interim/chr9_filtered.vcf data/interim/chr10_filtered.vcf data/interim/chr11_filtered.vcf data/interim/chr12_filtered.vcf data/interim/chr13_filtered.vcf data/interim/chr14_filtered.vcf data/interim/chr15_filtered.vcf data/interim/chr16_filtered.vcf data/interim/chr17_filtered.vcf data/interim/chr18_filtered.vcf data/interim/chr19_filtered.vcf data/interim/chr20_filtered.vcf data/interim/chr21_filtered.vcf data/interim/chr22_filtered.vcf -o data/interim/merged.vcf 


In [127]:
# plink2 --vcf  ALL.chr1.shapeit2_integrated_v1a.GRCh38.20181129.phased.vcf.gz --snps-only --maf 0.05 --geno 0.1  --mind 0.05  --recode vcf --out test

# ####### helper method for fastq2vcf ######
# def fastq2bam(**cfg):
#     lines = ''
#     for k, v in cfg['fastq2bam'].items():
#         if isinstance(v, list):
#             line = k + ' ' + ' '.join(v)
#         else:
#             line = k + ' ' + v
#         lines += line + ' '

        
        
# def bam2vcf(**cfg):
#     lines = ''
#     for k, v in cfg['bam2vcf'].items():
#         if isinstance(v, list):
#             line = k + ' ' + ' '.join(v)
#         else:
#             line = k + ' ' + v
#         lines += line + ' '
# ############ end of helper method ########

# def fastq2vcf(**cfg):

'3'

# command line

In [40]:
import os
import shutil
import shlex
import sys
import subprocess as sp

In [141]:
def pca_pruning(**proc_cfg):
    import shlex
    import subprocess as sp
    
    lines = ''
    for k, v in proc_cfg['pca_pruning'].items():
        line = k + ' ' + v
        lines += line + ' '
    cmd = 'plink2 ' + lines
#     print(cmd)
    #####
#     cmd_lst = shlex.split(cmd)
#     proc = sp.Popen(cmd_lst, stdout=sp.PIPE, stderr=sp.PIPE)
    #####
    return cmd

def pca_conduct(**proc_cfg):
    import shlex
    import subprocess as sp

    lines = ''
    for k, v in proc_cfg['pca_conduct'].items():
        line = k + ' ' + v
        lines += line + ' '
    cmd = 'plink2 ' + lines
#     print(cmd)
    #####
#     cmd_lst = shlex.split(cmd)
#     proc = sp.Popen(cmd_lst, stdout=sp.PIPE, stderr=sp.PIPE)
    #####
    return cmd

def plot_pca():
    import pandas as pd
    import numpy as np
    from matplotlib import pyplot as plt
    # will make it generic as development
    val_path = '../data/interim/pca.eigenval'
    vec_path = '../data/interim/pca.eigenvec'

    vec = pd.read_csv(vec_path, header = None, sep = " ")
    val = pd.read_csv(val_path, header = None, sep = " ")
    vec = vec.loc[:, 1:].set_index(1)

    plt.scatter(x = vec.loc[:, 2], y = vec.loc[:, 3], color = 'r', marker = '.')
    plt.title('PCA of 2 PC')
    plt.xlabel('1st PC')
    plt.ylabel('2nd PC')
    
    plt.savefig('pca_plot')
    
def process_data(**proc_cfg):
    pca_pruning(**proc_cfg);
    pca_conduct(**proc_cfg);
    
    # will need to take in/out path
    plot_pca();


In [134]:
proc_cfg = json.load(open('../config/process-params.json'))
proc_cfg
print(pca_pruning(**proc_cfg))
print(pca_conduct(**proc_cfg))


plink2 --vcf data/interim/merged.vcf --double-id  --allow-extra-chr  --set-missing-var-ids @:# --indep-pairwise 50 10 0.1 --out data/interim/pca 
plink2 --vcf data/interim/merged.vcf --double-id  --allow-extra-chr  --set-missing-var-ids @:# --extract data/interim/pca.prune.in --make-bed  --pca  --out data/interim/pca 


plink2 --vcf ../data/test/chr22_test.vcf.gz --double-id  --allow-extra-chr  --set-missing-var-ids @:# --extract ../data/interim/pca.prune.in --make-bed  --pca  --out ../data/interim/pca

In [144]:
# cmd = 'python --version'


In [92]:
# cmd_lst = shlex.split(cmd)
# cmd_lst


In [93]:
# sp.call(cmd_lst)

In [94]:
# proc = sp.Popen(cmd_lst, stdout=sp.PIPE, stderr=sp.PIPE)

In [95]:
# proc.stdout.read()

In [96]:
# os.path.join('..', 'data/raw', 'file')